In [1]:
from oo_cqed_rhf import CQEDRHFCalculator
import numpy as np
import psi4

In [9]:
lambda_vector = [0.0, 0.0,0.13041133]
# molecule string for H2O
molecule_string = """
0 1
    O      0.000000000000   0.000000000000  -0.068516219320
    H      0.000000000000  -0.790689573744   0.543701060715
    H      0.000000000000   0.790689573744   0.543701060715
no_reorient
no_com
symmetry c1
"""

# options for H2O
psi4_options = {
    "basis": "cc-pVDZ",
    "save_jk": True,
    "scf_type": "pk",
    "e_convergence": 1e-12,
    "d_convergence": 1e-12,
}


In [10]:
calc = CQEDRHFCalculator(lambda_vector, molecule_string, psi4_options)
calc.calc_cqed_rhf_energy()


Scratch directory: /tmp/
   => Libint2 <=

    Primary   basis highest AM E, G, H:  6, 6, 3
    Auxiliary basis highest AM E, G, H:  7, 7, 4
    Onebody   basis highest AM E, G, H:  -, -, -
    Solid Harmonics ordering:            Gaussian

*** tstart() called on CHEM9QDFT72ALT
*** at Tue Jun 17 17:11:35 2025

   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   198 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/cc-pvdz.gbs 
    atoms 2-3 entry H          line    22 file /Users/jfoley19/miniconda3/envs/psi4_new/share/psi4/basis/cc-pvdz.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         ----------------------------------------------

In [12]:
expected_h2o_e = -76.016355284146
#assert np.isclose(expected_h2o_e, calc.cqed_rhf_energy, 1e-9, 1e-9)
calc.summary()

RHF Energy:           -76.02141845 Ha
CQED-RHF Energy:      -75.98720569 Ha
Dipole Energy:        0.00025199 Ha
Nuclear Repulsion:    8.80146556 Ha
Dipole Moment:        [-1.37781592e-16  2.88657986e-15  8.46933549e-01]


In [5]:
calc.compute_nuclear_repulsion_gradient()
calc.compute_one_electron_integral_gradient_terms()
calc.compute_two_electron_integral_gradient_terms()
calc.compute_fock_matrix_term()

In [6]:
calc.calc_scf_gradient(qed_wfn=True)


*** tstart() called on CHEM9QDFT72ALT
*** at Tue Jun 17 17:10:19 2025


         ------------------------------------------------------------
                                   SCF GRAD                          
                          Rob Parrish, Justin Turney,                
                       Andy Simmonett, and Alex Sokolov              
         ------------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c1
    Full point group: C2v

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center              X                  Y                   Z               Mass       
    ------------   -----------------  -----------------  -----------------  -----------------
         O            0.000000000000     0.000000000000    -0.068516219320    15.994914619570
         H            0.000000000000    -0.790689573744     0.543701060715     1.007825032230
         H            0.000000000000     0.790689573744     0.54

array([[-3.69561367e-17,  9.32519913e-16, -6.01300796e-02],
       [-1.48146760e-17, -4.33923429e-02,  3.00650398e-02],
       [ 5.17708127e-17,  4.33923429e-02,  3.00650398e-02]])

r repulsion =    8.801465564563406

  ==> Basis Set <==

  Basis Set: CC-PVDZ
    Blend: CC-PVDZ
    Number of shells: 12
    Number of basis functions: 24
    Number of Cartesian functions: 25
    Spherical Harmonics?: true
    Max angular momentum: 2

  ==> DirectJKGrad: Integral-Direct SCF Gradients <==

    Gradient:                    1
    J tasked:                  Yes
    K tasked:                  Yes
    wK tasked:                  No
    Integrals threads:           1
    Schwarz Cutoff:          1E-12


  -Total Gradient:
     Atom            X                  Y                   Z
    ------   -----------------  -----------------  -----------------
       1       -0.000000000000     0.000000000000    -0.060130079561
       2       -0.000000000000    -0.043392342878     0.030065039780
       3        0.000000000000     0.043392342878     0.030065039780


*** tstop() called on CHEM9QDFT72ALT at Tue Jun 17 17:10:19 2025
Module time:
	user time   =       0.04 seconds =       

In [7]:
print(calc.scf_grad)
print("Shape of nuclear repulsion gradient")
print(np.shape(calc.nuclear_repulsion_gradient))
print("Shape of Overlap Gradient")
print(np.shape(calc.overlap_gradient))
print("Shape of T gradient")
print(np.shape(calc.kinetic_gradient))
print("Shape of V gradient")
print(np.shape(calc.potential_gradient))
print("Shape of J gradient")
print(np.shape(calc.J_gradient))
print("Shape of K gradient")
print(np.shape(calc.K_gradient))
el_grads = calc.overlap_gradient + calc.kinetic_gradient + calc.potential_gradient + calc.J_gradient + calc.K_gradient 
my_grad = calc.nuclear_repulsion_gradient + el_grads.reshape(3,3)
print("My gradient")
print(my_grad)


[[-3.69561367e-17  9.32519913e-16 -6.01300796e-02]
 [-1.48146760e-17 -4.33923429e-02  3.00650398e-02]
 [ 5.17708127e-17  4.33923429e-02  3.00650398e-02]]
Shape of nuclear repulsion gradient
(3, 3)
Shape of Overlap Gradient
(9,)
Shape of T gradient
(9,)
Shape of V gradient
(9,)
Shape of J gradient
(9,)
Shape of K gradient
(9,)
My gradient
[[-3.69561367e-17 -1.40084745e-16 -6.01300796e-02]
 [-1.48146760e-17 -4.33923429e-02  3.00650398e-02]
 [ 5.17708127e-17  4.33923429e-02  3.00650398e-02]]


In [ ]:
#print(calc.scf_grad -gradient)

In [ ]:
calc.calc_quadrupole_gradient()
# last output

#[[ 1.32601432e-18  9.99294165e-19 -2.77573793e-03]
# [ 1.03219418e-19  2.17498212e-04  1.62681438e-03]
# [-1.42923374e-18 -2.17498212e-04  1.62681438e-03]]

In [ ]:
calc.calc_numerical_gradient()

In [ ]:
calc.numerical_scf_gradient.reshape(3,3)-calc.scf_grad

In [ ]:
print("O_DSE Analytic GRAD")
print(calc.numerical_o_dse_gradient.reshape(3,3))


print("O_DSE Numeric GRAD")
print(calc.o_dse_gradient.reshape(3,3))

In [ ]:
calc.calc_dipole_dipole_gradient()

In [ ]:
print("K_DSE GRAD")
print(calc.numerical_K_dse_gradient.reshape(3,3))

print("ANALYTIC K_DSE GRAD")
print(calc.K_dse_gradient.reshape(3,3))




In [ ]:

print("E GRAD")
print(calc.numerical_energy_gradient.reshape(3,3))

sum_grad = calc.scf_grad + calc.numerical_o_dse_gradient.reshape(3,3) + calc.numerical_K_dse_gradient.reshape(3,3)

print("SUM GRAD")
print(sum_grad)

In [ ]:
total_gradient = calc.scf_grad + calc.K_dse_gradient.reshape(3,3) + calc.o_dse_gradient.reshape(3,3)
print(total_gradient)

# The numerical gradient for water in a cc-pVDZ basis set with this geometry and lambda = 0, 0, 0.05 is:

[[ 2.25601351e-10 -2.63201576e-10 -6.18353559e-02]

 [ 7.52004504e-11 -4.35898538e-02  3.09176777e-02]
 
 [ 0.00000000e+00  4.35898541e-02  3.09176786e-02]]

In [ ]:
_expected_gradient = np.array(
[[ 2.25601351e-10, -2.63201576e-10, -6.18353559e-02],
 [ 7.52004504e-11, -4.35898538e-02,  3.09176777e-02],
 [ 0.00000000e+00,  4.35898541e-02,  3.09176786e-02]])

print(_expected_gradient-calc.numerical_energy_gradient.reshape(3,3))